In [40]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np

In [58]:
soup = BeautifulSoup(requests.get('https://en.wikipedia.org/wiki/List_of_largest_companies_by_revenue').text,'lxml')

table = soup.find('table')

body = table.find('tbody')

body_detail = body.find_all('tr')

data = []
for row in body_detail:
    cols = row.find_all('td')
    arr = []
    for col in cols:
        arr.append(col.text.strip())
        if (col.get('data-sort-value') == "No"):
            arr.append("No") 
        elif(col.get('data-sort-value') == "Yes"):
            arr.append("Yes")
    data.append(arr)

df = pd.DataFrame(data, columns=['1','2','3','4','5','6','7','8','9'])
df.drop([0,1], inplace=True)

df.reset_index(drop=True,inplace=True)

df.drop(columns=['7','9'], inplace=True)

df.rename(columns={'1':'Name','2':'Industry','3':'Revenue','4':'Profit','5':'Employees','6':'Country','8':'State-owned'}, inplace=True)
df.rename(columns={'Revenue':'Revenue (USD millions)','Profit':'Profit (USD millions)'}, inplace=True)

df['Revenue (USD millions)'] = df['Revenue (USD millions)'].str.replace('$','')
df['Profit (USD millions)'] = df['Profit (USD millions)'].str.replace('$','')

df['Profit (USD millions)'] = df['Profit (USD millions)'].replace('...', np.nan)

df['Revenue (USD millions)'] = df['Revenue (USD millions)'].str.replace(',','')
df['Profit (USD millions)'] = df['Profit (USD millions)'].str.replace(',','')
df['Employees'] = df['Employees'].str.replace(',','')

df['Employees'] = df['Employees'].astype(int)
df['Revenue (USD millions)'] = df['Revenue (USD millions)'].astype(int)

df['Profit (USD millions)'] = df['Profit (USD millions)'].str.replace('−','-')
df['Profit (USD millions)'] = df['Profit (USD millions)'].str.replace('–','-')

df['Profit (USD millions)'] = pd.to_numeric(df['Profit (USD millions)'], errors='coerce')


df.to_csv('largest_companies.csv', index=False)